In [1]:
# NOTEBOOK: CRAWL AIR QUALITY (CAMS)
# CELL 1: Khởi tạo

import pandas as pd
import os
import glob
import openmeteo_requests
import requests_cache
from retry_requests import retry
from datetime import datetime
import time

print("Cell 1: Môi trường sẵn sàng.")

Cell 1: Môi trường sẵn sàng.


In [2]:
# CELL 2: Đọc file Metadata

metadata_filename = "stations_metadata.csv"

if not os.path.exists(metadata_filename):
    raise FileNotFoundError(f"LỖI: File '{metadata_filename}' không tồn tại. Vui lòng tạo file này trước.")
else:
    df_metadata = pd.read_csv(metadata_filename)
    print(f"Đã đọc thành công thông tin của {len(df_metadata)} trạm từ '{metadata_filename}'.")

Đã đọc thành công thông tin của 31 trạm từ 'stations_metadata.csv'.


In [3]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 3: Tải và Lưu file Air Quality Raw (TỪ 2022)

if 'df_metadata' not in locals():
    print("Vui lòng chạy Cell 2 trước.")
else:
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    aq_output_dir = "hanoi_air_quality_from04Aug_cams_raw"
    if not os.path.exists(aq_output_dir): os.makedirs(aq_output_dir)

    # --- KHOẢNG THỜI GIAN THỰC TẾ CÓ DỮ LIỆU LÀ TỪ NGÀY 04 THÁNG 8 NĂM 2022---
    START_DATE_FIXED = "2022-08-02"
    END_DATE_FIXED = "2025-10-10"
    # --------------------------------------------

    print(f"\nBắt đầu tải dữ liệu Air Quality cho {len(df_metadata)} vị trí từ {START_DATE_FIXED} đến {END_DATE_FIXED}...")

    for index, station in df_metadata.iterrows():
        loc_id = station['location_id']
        lat = station['lat']
        lon = station['lon']
        
        output_filename = os.path.join(aq_output_dir, f"air_quality_cams_{loc_id}.csv")
        if os.path.exists(output_filename):
            print(f"\n--- File cho trạm ID: {loc_id} đã tồn tại. Bỏ qua. ---")
            continue

        print(f"\n--- Đang lấy dữ liệu Air Quality cho vị trí của trạm ID: {loc_id} ---")

        url = "https://air-quality-api.open-meteo.com/v1/air-quality"
        params = {
            "latitude": lat, "longitude": lon,
            "start_date": START_DATE_FIXED, "end_date": END_DATE_FIXED,
            "hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
        }
        
        try:
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]
            
            hourly = response.Hourly()
            hourly_data = {"datetime": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )}

            for i, var_name in enumerate(params["hourly"]):
                values = hourly.Variables(i).ValuesAsNumpy()
                hourly_data[f"{var_name}_cams"] = values[:len(hourly_data["datetime"])]

            df_aq_cams = pd.DataFrame(data=hourly_data)
            df_aq_cams.to_csv(output_filename, index=False, encoding='utf-8-sig')
            
            print(f"  -> Đã lưu thành công {len(df_aq_cams)} giờ dữ liệu vào file {output_filename}")
        except Exception as e:
            print(f"  -> Đã xảy ra lỗi: {e}")
            
        finally:
            # --- LOGIC MỚI: NGHỈ SAU MỖI LẦN GỌI ---
            # Thêm một khoảng nghỉ 4 giây sau mỗi lần gọi API, dù thành công hay thất bại
            # để đảm bảo không vượt quá giới hạn.
            print("    -> Nghỉ 4 giây...")
            time.sleep(4)
            # ----------------------------------------

    print("\n\n---> HOÀN TẤT TẢI DỮ LIỆU AIR QUALITY (CAMS) <---")


Bắt đầu tải dữ liệu Air Quality cho 31 vị trí từ 2022-08-02 đến 2025-10-10...

--- File cho trạm ID: 2539 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 7441 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 1285357 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161290 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161291 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161292 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161293 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161294 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161295 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161296 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161298 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161299 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161300 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161301 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161303 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161304 đã tồn tại. Bỏ qua. ---

--- File cho trạm ID: 2161306 đã tồn tại. Bỏ q

In [4]:
# NOTEBOOK OPEN-METEO AIR QUALITY
# CELL 4: Gộp các file Air Quality Raw

aq_output_dir = "hanoi_air_quality_from04Aug_cams_raw"

if not os.path.exists(aq_output_dir):
    print(f"Thư mục '{aq_output_dir}' không tồn tại. Vui lòng chạy Cell 3 trước.")
else:
    # 1. Tìm tất cả các file .csv trong thư mục
    all_csv_files = glob.glob(os.path.join(aq_output_dir, "*.csv"))
    
    if not all_csv_files:
        print("Không tìm thấy file CSV nào để gộp.")
    else:
        print(f"Tìm thấy {len(all_csv_files)} file CAMS CSV. Bắt đầu gộp...")
        
        # 2. Đọc, thêm location_id và gộp tất cả các file
        all_data_frames = []
        for f in all_csv_files:
            df = pd.read_csv(f)
            # Lấy location_id từ tên file
            loc_id = int(os.path.basename(f).replace('air_quality_cams_', '').replace('.csv', ''))
            df['location_id'] = loc_id
            all_data_frames.append(df)
            
        final_df = pd.concat(all_data_frames, ignore_index=True)
        
        # 3. Xử lý và sắp xếp lần cuối
        final_df['datetime'] = pd.to_datetime(final_df['datetime'])
        final_df = final_df.sort_values(by=['location_id', 'datetime'])
        
        # 4. Lưu file tổng hợp cuối cùng
        final_output_filename = "hanoi_air_quality_from04Aug_CAMS_COMBINED.csv"
        final_df.to_csv(final_output_filename, index=False, encoding='utf-8-sig')
        
        print("\n---> QUÁ TRÌNH GỘP HOÀN TẤT <---")
        print(f"Tổng cộng có {len(final_df)} dòng dữ liệu.")
        print(f"Dữ liệu CAMS đã được gộp và lưu vào file '{final_output_filename}'")
        print("\nThông tin chi tiết của DataFrame cuối cùng:")
        final_df.info()

Tìm thấy 31 file CAMS CSV. Bắt đầu gộp...

---> QUÁ TRÌNH GỘP HOÀN TẤT <---
Tổng cộng có 867504 dòng dữ liệu.
Dữ liệu CAMS đã được gộp và lưu vào file 'hanoi_air_quality_from04Aug_CAMS_COMBINED.csv'

Thông tin chi tiết của DataFrame cuối cùng:
<class 'pandas.core.frame.DataFrame'>
Index: 867504 entries, 55968 to 531695
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype              
---  ------                 --------------   -----              
 0   datetime               867504 non-null  datetime64[ns, UTC]
 1   pm10_cams              866016 non-null  float64            
 2   pm2_5_cams             866016 non-null  float64            
 3   carbon_monoxide_cams   866016 non-null  float64            
 4   nitrogen_dioxide_cams  866016 non-null  float64            
 5   sulphur_dioxide_cams   866016 non-null  float64            
 6   ozone_cams             866016 non-null  float64            
 7   location_id            867504 non-null  int64            